In [87]:
import pandas as pd
from ydata_profiling import ProfileReport

In [88]:
df = pd.read_csv('csv/uspevaemost.csv', encoding='cp1251')

In [89]:
df = df[df['Статус'] == 'Отчисленный']

df = df[df['Квалификация'] != 'Слушатель программ довузовской подготовки']

In [90]:
df.drop_duplicates(inplace=True)

df = df[~(df['Оценка'] == 'неудовлетворительно') | ~(df.duplicated(subset=df.columns.difference(['Оценка']))) ]

df['Оценка'] = df['Оценка'].map({
    'отлично': 5,
    'хорошо': 4,
    'удовлетворительно': 3,
    'зачтено': 2, 
    'незачет': 1 
})

df_bez_D = df.drop_duplicates(subset=df.columns.difference(['Оценка']), keep=False)

df_s_D = df.merge(df_bez_D, indicator=True, how='outer').loc[lambda x: x['_merge'] == 'left_only'].drop(columns=['_merge']).drop_duplicates()

In [91]:
max_indices = df_s_D.groupby(['Студент', 'Группа', 'Дисциплина', 'Семестр', 'Учебный год', 'Специальность', 'Форма обучения', 'Квалификация', 'Статус'])['Оценка'].idxmax(skipna=True)
df_max = df_s_D.loc[max_indices]


min_indices = df_s_D.groupby(['Студент', 'Группа', 'Дисциплина', 'Семестр', 'Учебный год', 'Специальность', 'Форма обучения', 'Квалификация', 'Статус'])['Оценка'].idxmin(skipna=True)
df_min = df_s_D.loc[min_indices]


In [92]:
df_min = df_min.groupby(['Студент', 'Группа', 'Семестр', 'Учебный год', 'Специальность', 'Форма обучения', 'Квалификация', 'Статус'], as_index=True)['Оценка'].value_counts().unstack(fill_value=0)
df_max = df_max.groupby(['Студент', 'Группа', 'Семестр', 'Учебный год', 'Специальность', 'Форма обучения', 'Квалификация', 'Статус'], as_index=True)['Оценка'].value_counts().unstack(fill_value=0)
df_bez_D = df_bez_D.groupby(['Студент', 'Группа', 'Семестр', 'Учебный год', 'Специальность', 'Форма обучения', 'Квалификация', 'Статус'], as_index=True)['Оценка'].value_counts().unstack(fill_value=0)

In [93]:

df = pd.merge(df_bez_D, df_max, on=['Студент','Группа','Семестр','Учебный год','Специальность','Форма обучения','Квалификация','Статус'], how='outer')
df = pd.merge(df, df_min, on=['Студент','Группа','Семестр','Учебный год','Специальность','Форма обучения','Квалификация','Статус'], how='outer')

df.fillna(0, inplace=True)


In [94]:
df.rename(columns={'1_x': 'незачет', '2_x': 'зачет', '3_x': 'удовлетворительно', '4_x': 'хорошо', '5_x': 'отлично', '2_y': 'зачет испр', '3_y': 'удовлетворительно испр', '4_y': 'хорошо испр', '5_y': 'отлично испр', '1_y': 'незачет до испр', 2: 'зачет до испр', 3: 'удовлетворительно до испр', 4: 'хорошо до испр'}, inplace=True)

In [95]:

phrases = ['незачет', 'зачет', 'удовлетворительно', 'хорошо',
         'отлично', 'зачет испр', 'удовлетворительно испр',
         'хорошо испр', 'отлично испр', 'незачет до испр',
         'зачет до испр', 'удовлетворительно до испр', 'хорошо до испр']

df = df.sort_values(by=['Студент', 'Семестр'])
for phrase in phrases:
    df[f'Накоп {phrase}'] = df.groupby('Студент')[f'{phrase}'].cumsum()


In [96]:
df['незачет'] = df['незачет'].apply(lambda x: 1 if x >= 1 else 0)

df['отчислен'] = df.groupby('Студент')['незачет'].transform('max')

In [97]:
df = df[df['незачет'] < 1]
df = df[df['Накоп незачет'] < 1]



In [98]:
df.to_csv('tempData.csv', index=True)
df = pd.read_csv('tempData.csv')

df['Квалификация'] = df['Квалификация'].map({
    'Аспирант': 3,
    'Магистр': 2,
    'Специалист': 1,
    'Бакалавр': 0,
})

df['Форма обучения'] = df['Форма обучения'].map({
    'Заочная': 2,
    'Очная': 0,
   'Очно-заочная': 1
})

In [99]:

df.drop('Статус', axis=1, inplace=True)
df.drop('Студент', axis=1, inplace=True)
df.drop('Группа', axis=1, inplace=True)
df.drop('Специальность', axis=1, inplace=True)
df.drop('Учебный год', axis=1, inplace=True)
# df.drop('незачет', axis=1, inplace=True)
# df.drop('Накоп незачет', axis=1, inplace=True)


In [100]:
df.to_csv('csv/dataset.csv', index=False)




In [102]:
df.head()

,Семестр,Форма обучения,Квалификация,незачет,зачет,удовлетворительно,хорошо,отлично,зачет испр,удовлетворительно испр,...,Накоп отлично,Накоп зачет испр,Накоп удовлетворительно испр,Накоп хорошо испр,Накоп отлично испр,Накоп незачет до испр,Накоп зачет до испр,Накоп удовлетворительно до испр,Накоп хорошо до испр,отчислен
0,1,2,0,0,6.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2,2,0,0,5.0,1.0,0.0,2.0,0.0,0.0,...,3.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0
2,3,2,0,0,2.0,0.0,2.0,1.0,0.0,0.0,...,4.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0
3,4,2,0,0,4.0,0.0,0.0,2.0,0.0,0.0,...,6.0,0.0,0.0,2.0,1.0,0.0,0.0,3.0,0.0,0
4,5,2,0,0,5.0,0.0,2.0,0.0,0.0,0.0,...,6.0,0.0,0.0,3.0,1.0,0.0,0.0,4.0,0.0,0


In [103]:
profile = ProfileReport(df, title="Profiling Report")
profile.to_file("your_reportXD1.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\fluke\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ydata_profiling\model\typeset.py:125: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  not pdt.is_categorical_dtype(series)
C:\Users\fluke\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ydata_profiling\model\typeset.py:125: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  not pdt.is_categorical_dtype(series)
C:\Users\fluke\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ydata_profiling\model\typeset.py:125: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, 

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\fluke\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\fluke\AppData\Local\Temp\ipykernel_11172\749556828.py", line 2, in <module>
    profile.to_file("your_reportXD1.html")
  File "C:\Users\fluke\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\typeguard\__init__.py", line 1033, in wrapper
    retval = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fluke\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ydata_profiling\profile_report.py", line 355, in to_file
    data = self.to_html()
           ^^^^^^^^^^^^^^
  File "C:\Users\fluke\AppData\Local\Packages\Pytho

df.to_csv('csv/uspevaemost1000.csv', index=False)